<a href="https://colab.research.google.com/github/RAMGUNJAN/ERAV1_SESSION9/blob/main/session_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms



In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Calculate the mean of the CIFAR-10 dataset
data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=False)
mean_value = 0.0
for images, _ in data_loader:
    mean_value = images.mean(dim=(0, 2, 3))

# Define Albumentations transformations
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.CoarseDropout(max_holes=1, max_height=16, max_width=16, min_holes=1, min_height=16, min_width=16,
                    fill_value=tuple(mean_value.tolist()), mask_fill_value=None),
    ToTensorV2(),
], additional_targets={'image2': 'image'})

# Apply the transformations to the CIFAR-10 dataset
augmented_train_dataset = []
for image, label in train_dataset:
    image_np = np.transpose(image.numpy(), (1, 2, 0))  # Convert from (C, H, W) to (H, W, C)
    augmented = transform(image=image_np, image2=image_np)
    augmented_image = augmented["image"]
    augmented_label = label
    augmented_train_dataset.append((augmented_image, augmented_label))

# Apply the transformations to the test dataset as well
augmented_test_dataset = []
for image, label in test_dataset:
    image_np = np.transpose(image.numpy(), (1, 2, 0))  # Convert from (C, H, W) to (H, W, C)
    augmented = transform(image=image_np, image2=image_np)
    augmented_image = augmented["image"]
    augmented_label = label
    augmented_test_dataset.append((augmented_image, augmented_label))

# Create DataLoader for training and testing
batch_size = 64
train_loader = torch.utils.data.DataLoader(augmented_train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(augmented_test_dataset, batch_size=batch_size, shuffle=False)

# Now, 'train_loader' and 'test_loader' contain the augmented datasets that you can use for training and testing your model.



100%|██████████| 170498071/170498071 [00:06<00:00, 26952503.20it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
from google.colab import files
files.upload()

Saving modell.py to modell.py


{'modell.py': b'# -*- coding: utf-8 -*-\n"""model.ipynb\n\nAutomatically generated by Colaboratory.\n\nOriginal file is located at\n    https://colab.research.google.com/drive/1X9hX6zn4vsbh_GA7cygaDv2ruEIBKk2Z\n"""\n\nimport torch.nn as nn\nimport torch.nn.functional as F\n\nclass ConvBlock(nn.Module):\n    def __init__(self, in_channels, out_channels):\n        super(ConvBlock, self).__init__()\n        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)\n        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)\n        self.conv3 = nn.Conv2d(out_channels, out_channels, kernel_size=3, dilation=2)\n        self.bn1 = nn.BatchNorm2d(out_channels)\n        self.bn2 = nn.BatchNorm2d(out_channels)\n        self.bn3 = nn.BatchNorm2d(out_channels)\n        self.dropout = nn.Dropout(0.1)\n\n    def forward(self, x):\n        x = F.relu(self.bn1(self.conv1(x)))\n        x = F.relu(self.bn2(self.conv2(x)))\n        x = F.relu(self.bn3(self.con

In [4]:
import modell
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

model = modell.MyCNN(10).to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 32, 32]             280
       BatchNorm2d-2           [-1, 10, 32, 32]              20
            Conv2d-3           [-1, 10, 32, 32]             910
       BatchNorm2d-4           [-1, 10, 32, 32]              20
            Conv2d-5           [-1, 10, 28, 28]             910
       BatchNorm2d-6           [-1, 10, 28, 28]              20
           Dropout-7           [-1, 10, 28, 28]               0
         ConvBlock-8           [-1, 10, 28, 28]               0
            Conv2d-9           [-1, 20, 28, 28]           1,820
      BatchNorm2d-10           [-1, 20, 28, 28]              40
           Conv2d-11           [-1, 20, 28, 28]           3,620
      BatchNorm2d-12           [-1, 20, 28, 28]              40
           Conv2d-13           [-1, 20, 24, 24]           3,620
      BatchNorm2d-14           [-1

In [5]:
from tqdm import tqdm
import torch.nn.functional as F

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        # get samples
        data, target = data.to(device), target.to(device)

        # Init
        optimizer.zero_grad()

        # Predict
        y_pred = model(data)

        # Calculate loss
        loss = F.cross_entropy(y_pred, target)
        train_losses.append(loss.item())

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Update pbar-tqdm
        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc=f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [7]:
import modell
EPOCHS=60

# Initialize the model, optimizer, and device as before
model = modell.MyCNN(10).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training loop
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


EPOCH: 0


Loss=1.6844738721847534 Batch_id=781 Accuracy=33.83: 100%|██████████| 782/782 [00:17<00:00, 44.97it/s]



Test set: Average loss: 1.6657, Accuracy: 3693/10000 (36.93%)

EPOCH: 1


Loss=1.8205900192260742 Batch_id=781 Accuracy=45.27: 100%|██████████| 782/782 [00:17<00:00, 44.12it/s]



Test set: Average loss: 1.5421, Accuracy: 4248/10000 (42.48%)

EPOCH: 2


...


Test set: Average loss: 0.9311, Accuracy: 6910/10000 (69.10%)

EPOCH: 58


Loss=0.6994633078575134 Batch_id=781 Accuracy=81.50: 100%|██████████| 782/782 [00:17<00:00, 44.90it/s]



Test set: Average loss: 0.9400, Accuracy: 6912/10000 (69.12%)

EPOCH: 59


Loss=0.8398635387420654 Batch_id=781 Accuracy=81.65: 100%|██████████| 782/782 [00:17<00:00, 44.96it/s]



Test set: Average loss: 0.9490, Accuracy: 6887/10000 (68.87%)

